In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.models as models
import matplotlib.pyplot as plt

In [ ]:
#model = models.vgg11_bn()
#model

In [ ]:
# Mean and Standard Deviation for normalisation of images.
# Values are taken from existing image classification models 
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Define training transformation for data augmentation, including
# changing BCS, random rotation, random horizontal and vertical flips
# and resize image to (224,224) and normalisation by given mean and std
trainTransform = transforms.Compose(
    [transforms.RandomResizedCrop(224),
     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
     transforms.RandomHorizontalFlip(),
     transforms.RandomVerticalFlip(),
     transforms.RandomRotation(30),
     transforms.ToTensor(),
     transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

# Define validation and testing transformation, including
# resizing image to (224,224) and normalisation by given mean and std
valTransform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

testTransform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
    ])

In [ ]:
# Downloading datasets by pre-defined splits
trainData = datasets.Flowers102(root='data', split = 'train', download=True, transform=trainTransform)
valData = datasets.Flowers102(root='data', split = 'val', download=True, transform=valTransform)
testData = datasets.Flowers102(root='data', split = 'test', download=True, transform=testTransform)

In [ ]:
# Displaying a few images from the training dataset
figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(trainData), size=(1,)).item()
    img, label = trainData[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.axis("off")
    plt.imshow(img.permute(1,2,0), cmap="gray")
plt.show()

In [ ]:
class MyCNN(nn.Module):

    def __init__(self):
      super(MyCNN, self).__init__()
      self.convLayer = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(),
        nn.BatchNorm2d(128),
        nn.MaxPool2d(kernel_size=2, stride=2),

        nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
        nn.ReLU(),
        nn.BatchNorm2d(256),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Dropout(0.5)
      )
      
      self.classLayer = nn.Sequential(
          nn.Flatten(),

          nn.Linear(in_features=50176, out_features=2048, bias=True),
          nn.Dropout(p=0.5),
          nn.ReLU(),
          

          nn.Linear(in_features=2048, out_features=512, bias=True),
          nn.Dropout(p=0.5),
          nn.ReLU(),
          

          nn.Linear(in_features=512, out_features=102, bias=True)
      )

    def forward(self, x):
      x = self.convLayer(x)
      x = self.classLayer(x)

      return x




In [ ]:
# Hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 500
LR = 0.0001
WD = 0.0001

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

# Model
model = MyCNN()
model = model.to(device)

# Optimizer and Scheduler for adaptive learning rate
optimizer = optim.Adam(model.parameters(), lr = LR, weight_decay = WD)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor = 0.5, patience=5)

# Loss Function
loss = nn.CrossEntropyLoss()

In [ ]:
# Creating dataloaders for iteration into batches and assigning 4 parrallel computations
trainLoader = DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
valLoader = DataLoader(valData, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)
testLoader = DataLoader(testData, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)

In [ ]:
# Defining function for model training
losses = []
accuracy = []
def trainModel(dataloader, model, lossFunction, optimizer):
    model.train()
    currentLoss = 0.0
    correct = 0
    total = 0
    epochLoss = 0

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        total += y.size(0)

        optimizer.zero_grad()
        pred = model(X)
        loss = lossFunction(pred, y)
        loss.backward()
        optimizer.step()

        currentLoss += loss.item()
        epochLoss += lossFunction(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    epochLoss = epochLoss/total
    correct = correct/total

    losses.append(epochLoss)
    accuracy.append(correct * 100)
    print(f'Training: Accuracy {correct * 100:>0.1f}%, Loss: {currentLoss / len(dataloader):.5f}, Epoch Loss: {epochLoss:.5f}')



In [ ]:
# Defining function to calculate validation accuracy
def validateModel(dataloader, model, lossFunction):
    model.eval()
    total = 0
    correct = 0
    epochLoss = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            total += y.size(0)
            epochLoss += lossFunction(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    epochLoss = epochLoss/total
    correct = (correct/total) * 100

    print(f"Validation: Accuracy: {(correct):>0.1f}%, Avg loss: {epochLoss:>8f} \n")
    return epochLoss, correct

In [ ]:
# Applying training function and output accuracy at each epoch iteration
bestValAccuracy = 0.0
bestValLoss = float('inf')
bestEpoch = 0

valLosses = []
valAccuracies = []

for epoch in range(NUM_EPOCHS):
    print(f'Epoch {epoch+1}:')
    trainModel(trainLoader, model, loss, optimizer)
    valLoss, valAccuracy = validateModel(valLoader, model, loss)
    valLosses.append(valLoss)
    valAccuracies.append(valAccuracy)
    scheduler.step(valLoss)

    if (valLoss < bestValLoss):
        bestValLoss = valLoss
        bestValAccuracy = valAccuracy
        bestEpoch = epoch+1

print(f'Best Accuracy: {bestValAccuracy}. Best Loss: {bestValLoss} Best Epoch: {bestEpoch}')

Validation Accuract at 0.0001 learning rate: 28%